In [18]:
import os
sep = os.sep
import numpy as np
from testarch.unet import runs as ru
from testarch.unet import runs as rm
import utils.auto_split as asp

In [16]:
from PIL import Image as IMG
from utils.img_utils import Image
from sklearn.metrics import f1_score
from scipy.ndimage.morphology import distance_transform_edt

In [19]:
unet_drive = [ru.DRIVE_1_100_1, ru.DRIVE_1_1, ru.DRIVE_WEIGHTED]
unet_stare = [ru.STARE_1_100_1, ru.STARE_1_1, ru.STARE_WEIGHTED]
unet_wide = [ru.WIDE_1_100_1, ru.WIDE_1_1, ru.WIDE_WEIGHTED]
unet_chasedb = [ru.CHASEDB_1_100_1, ru.CHASEDB_1_1, ru.CHASEDB_WEIGHTED]
unet_vevio_mosacis = [ru.VEVIO_MOSAICS_1_100_1, ru.VEVIO_MOSAICS_1_1, ru.VEVIO_MOSAICS_WEIGHTED]
unet_vevio_frames = [ru.VEVIO_FRAMES_1_100_1, ru.VEVIO_FRAMES_1_1, ru.VEVIO_FRAMES_WEIGHTED]

munet_drive = [rm.DRIVE_1_100_1, rm.DRIVE_1_1, rm.DRIVE_WEIGHTED]
munet_stare = [rm.STARE_1_100_1, rm.STARE_1_1, rm.STARE_WEIGHTED]
munet_wide = [rm.WIDE_1_100_1, rm.WIDE_1_1, rm.WIDE_WEIGHTED]
munet_chasedb = [rm.CHASEDB_1_100_1, rm.CHASEDB_1_1, rm.CHASEDB_WEIGHTED]
munet_vevio_mosaics = [rm.VEVIO_MOSAICS_1_100_1, rm.VEVIO_MOSAICS_1_1, rm.VEVIO_MOSAICS_WEIGHTED]
munet_vevio_frames =[rm.VEVIO_FRAMES_1_100_1, rm.VEVIO_FRAMES_1_1, rm.VEVIO_FRAMES_WEIGHTED]

In [54]:
datasets = [unet_drive, unet_stare, unet_wide, unet_chasedb, unet_vevio_mosacis, unet_vevio_frames]

In [ ]:
for d100, d11, dw in datasets:
    print('========================================================')
    test_files = []
    for split_file in os.listdir(d100['Dirs']['splits_json']):
        test_files += asp.load_split_json(d100['Dirs']['splits_json']+sep+split_file)['test']
    print('DATASET  FILE    F1_100_1 F1_1_1 F1_WEIGHTED DIFF')
    print('--------------------------------------------------------')
    for file in test_files:
        image = Image()
        image.load_file(d100['Dirs']['image'], file)
        image.load_ground_truth(d100['Dirs']['truth'], d100['Funcs']['truth_getter'])
        
        if d100['Dirs'].get('mask'):
            image.load_mask(d100['Dirs']['mask'], d100['Funcs']['mask_getter'])

        pred_file =  'pred_' + file.split('.')[0] + '.png'
        image.u_1_100 = np.array(IMG.open(d100['Dirs']['logs'] + sep + pred_file))
        image.u_1_1 = np.array(IMG.open(d11['Dirs']['logs'] + sep + pred_file))
        image.uw = np.array(IMG.open(dw['Dirs']['logs'] + sep + pred_file))
        
        image.u_1_100[image.u_1_100==255] = 1
        image.u_1_1[image.u_1_1==255] = 1
        image.uw[image.uw==255] = 1
        
        gt = image.ground_truth.copy()
        gt[gt==255] = 1
        edt_gt = distance_transform_edt(gt)

        edt = edt_gt.copy()
        edt[edt==0] = 1
        edt = 1/edt

        f1_u100 = f1_score(gt.flatten(), image.u_1_100.flatten(),\
             sample_weight=edt.flatten())
        f1_u1 = f1_score(gt.flatten(), image.u_1_1.flatten(),\
             sample_weight=edt.flatten())
        
        f1_uw = f1_score(gt.flatten(), image.uw.flatten(),\
             sample_weight=edt.flatten())
        print(d100['Dirs']['image'].split('\\')[1], file, round(f1_u100, 5), round(f1_u1, 5), round(f1_uw, 5), round(f1_u100-f1_u1, 5))

### SPLIT FOUND:  data\DRIVE\splits\UNET-DRIVE.json Loaded
DATASET  FILE    F1_100_1 F1_1_1 F1_WEIGHTED DIFF
--------------------------------------------------------
DRIVE 11_test.tif 0.75918 0.76592 0.74055 -0.00673
DRIVE 02_test.tif 0.8026 0.80034 0.78034 0.00226
DRIVE 15_test.tif 0.77177 0.76431 0.74729 0.00746
DRIVE 04_test.tif 0.77292 0.75983 0.75506 0.01309
DRIVE 20_test.tif 0.76748 0.77743 0.74044 -0.00995
DRIVE 01_test.tif 0.78303 0.79508 0.76395 -0.01204
DRIVE 19_test.tif 0.82024 0.83999 0.79424 -0.01975
DRIVE 08_test.tif 0.74354 0.73432 0.73032 0.00923
DRIVE 06_test.tif 0.74961 0.73165 0.73478 0.01796
DRIVE 18_test.tif 0.76602 0.77766 0.74713 -0.01165
DRIVE 14_test.tif 0.77621 0.77826 0.75487 -0.00206
DRIVE 03_test.tif 0.74088 0.72923 0.72442 0.01165
DRIVE 07_test.tif 0.76447 0.75824 0.75126 0.00623
DRIVE 09_test.tif 0.76779 0.75767 0.74253 0.01013
DRIVE 12_test.tif 0.78078 0.78223 0.76004 -0.00145
DRIVE 16_test.tif 0.80232 0.79854 0.77692 0.00378
DRIVE 05_test.tif 0.75486 0.